# Google Dataset Train npy Generation

In [36]:
!pip install ffmpeg

In [51]:
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
import random
import sys
import io
import os
import glob
import IPython
import wave
import pylab
import pandas as pd
from tf_utils import *
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

# Import files for trigger-word detection model
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\Arushi Gupta\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\Arushi Gupta\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.

During handling of the above exception, another exception occ

TypeError: can only concatenate str (not "list") to str

In [52]:
POSITIVE_DIRECTORY = "./raw_data/positive_data/"
BACKGROUND_DIRECTORY = "./raw_data/background_data/"
NEGATIVES_DIRECTORY = "./raw_data/google_dataset/"
NEGATIVES_TRUNCATED_DIRECTORY = "./raw_data/google_dataset_truncated/"
AUDIO_EXAMPLES_DIRECTORY = "./audio_examples/"
AUDIO_IGNORED_EXAMPLES_DIRECTORY = "./audio_ignored_examples/"
POSITIVE_EXAMPLE = "jh_1.wav"
AUDIO_EXAMPLE = "example_train.wav"
STUB_TRAIN_DIRECTORY = "./stub_data/XY_Train/"
STUB_DEV_DIRECTORY = "./stub_data/XY_Dev/"
STUB_MODEL = "./stub_data/models/tr_model.h5"
CONT_EXAMPLE_DIRECTORY = "./cont_example/"
DEV_DIRECTORY = "./raw_data/dev_dataset/"
DEV_CUT_DIRECTORY = "./raw_data/dev_dataset_cut/"
DEV_NPY_DIRECTORY = "./raw_data/dev_npy/"
DEV_GOOGLE_BASED_DATASET_CUT = "./raw_data/dev_google_based_dataset_cut/"
GOOGLE_DATASET_BASED_NPY = "./raw_data/google_dataset_based_npy/"

In [53]:
play(AudioSegment.from_file(DEV_GOOGLE_BASED_DATASET_CUT + "jh_1.wav"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-d1ba082e8718>", line 1, in <module>
    play(AudioSegment.from_file(DEV_GOOGLE_BASED_DATASET_CUT + "jh_1.wav"))
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\pydub\playback.py", line 53, in play
    playback = _play_with_simpleaudio(audio_segment)
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\pydub\playback.py", line 47, in _play_with_simpleaudio
    sample_rate=seg.frame_rate
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\simpleaudio\shiny.py", line 61, in play_buffer
    sample_rate)
ValueError: Weird sample rates are not supported.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.

ValueError: Weird sample rates are not supported.

In [54]:
Tx = 5490 # Based on created training example
n_freq = 129 # Based on created training example
Ty = 1369 # Based on model.summary() in 1.4 with shape := (Tx, n_freq)

## Some Original Functions

In [ ]:
# Original insert_ones(y,segment_end_ms)
# def insert_ones(y, segment_end_ms):
#     """
#     Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
#     should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
#     50 followinf labels should be ones.
    
    
#     Arguments:
#     y -- numpy array of shape (1, Ty), the labels of the training example
#     segment_end_ms -- the end time of the segment in ms
    
#     Returns:
#     y -- updated labels
#     """
    
#     # duration of the background (in terms of spectrogram time-steps)
#     segment_end_y = int(segment_end_ms * Ty / 10000.0)
#     print(segment_end_y)
    
#     # Add 1 to the correct index in the background label (y)
#     ### START CODE HERE ### (≈ 3 lines)
#     for i in range(segment_end_y + 1, segment_end_y + 51):
#         if i < Ty:
#             y[0, i] = 1
#     ### END CODE HERE ###
    
#     return y

In [ ]:
# Original create_training_example(id)
# def create_training_example(id):
#     """
#     Creates a training examples with a given background, positives, and negatives with id.
    
#     Arguments:
#     id -- an id is given such that the new files does not replace the previous files
    
#     Returns:
#     x -- the spectrogram of the training example
#     y -- the label at each time step of the spectrogram
#     """
#     global BACKGROUND_AUDIONAMES
#     global POSITIVES_AUDIONAMES
#     global NEGATIVES_AUDIONAMES
    
#     background = random_background(BACKGROUND_AUDIONAMES)
#     positives = random_positives(POSITIVES_AUDIONAMES)
#     negatives = random_negatives(NEGATIVES_AUDIONAMES)
    
#     # Make background quieter
#     background = background - 20
    
#     # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
#     y = np.zeros((1, Ty))

#     # Step 2: Initialize segment times as empty list (≈ 1 line)
#     previous_segments = []    
    
#     # Step 3: Loop over randomly selected "activate" clips and insert in background
#     for random_positive in positives:
#         if cannot_insert(random_positive, previous_segments):
#             return create_training_example(id)
#         # Insert the audio clip on the background
#         background, segment_time = insert_audio_clip(background, random_positive, previous_segments)
#         # Retrieve segment_start and segment_end from segment_time
#         segment_start, segment_end = segment_time
#         # Insert labels in "y"
#         y = insert_ones(y, segment_end_ms=segment_end)

#     # Step 4: Loop over randomly selected negative clips and insert in background
#     for random_negative in negatives:
#         if cannot_insert(random_negative, previous_segments):
#             return create_training_example(id)
#         # Insert the audio clip on the background 
#         background, _ = insert_audio_clip(background, random_negative, previous_segments)

#     # Export new training example 
#     background = background.set_channels(1)
#     background = background.set_frame_rate(123000)
    
#     TRAIN_PREFIX = "train_"
#     file_handle = background.export(AUDIO_EXAMPLES_DIRECTORY + TRAIN_PREFIX + str(id) + ".wav", format="wav")
#     # print("File (train_" + str(id) + ".wav) was saved in your directory.")

#     sample_rate, samples = wavfile.read(AUDIO_EXAMPLES_DIRECTORY + TRAIN_PREFIX + str(id) +".wav")
#     frequencies, times, x = signal.spectrogram(samples, sample_rate)
    
#     return frequencies, times, x, y

In [ ]:
# Original create_X_Y(size,start)
# def create_X_Y(size, start):
#     i, X, Y = 1, [], []
#     for i in range(start, start + size):
#         frequencies, times, x, y = create_training_example(i)
#         x = np.transpose(x)
#         y = np.transpose(y)
#         X.append(x)
#         Y.append(y)
#     return (np.array(X), np.array(Y))

## Helper Functions

In [55]:
def insert_ones_for_human_train_eg(y, segment_ends):
    """
    Update the label vector y. The labels of the 50 output steps strictly after the end of the segment 
    should be set to 1. By strictly we mean that the label of segment_end_y should be 0 while, the
    50 followinf labels should be ones.
    
    
    Arguments:
    y -- numpy array of shape (1, Ty), the labels of the training example
    segment_end_ms -- A list of end times segments of the dev file in ms
    
    Returns:
    y -- updated labels
    """

    # duration of the background (in terms of spectrogram time-steps)
    for segment_end_ms in segment_ends:
        segment_end_y = int(segment_end_ms * Ty / 10000.0)

        # Add 1 to the correct index in the background label (y)
        ### START CODE HERE ### (≈ 3 lines)
        for i in range(segment_end_y + 1, segment_end_y + 51):
             if i < Ty:
                 y[0, i] = 1
        ### END CODE HERE ###

    return y

In [56]:
def sec_to_ms(sec):
    return sec * 1000

In [57]:
def create_human_train_eg(name,id):
    """
    Creates dev_examples
    Arguments:
    name -- name prefixed to id
    id -- an id is given such that the new files does not replace the previous files
    
    Returns:
    x -- the spectrogram of the training example
    y -- the label at each time step of the spectrogram
    """
    global DEV_GOOGLE_BASED_DATASET_CUT
    
    # Step 1: Initialize y (label vector) of zeros (≈ 1 line)
    y = np.zeros((1, Ty))
    
    # Step 2: Import end time segments    
    segment_ends = pd.read_csv(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) + ".txt", header=None) # Read in the end_time_segments txt
    if not str(segment_ends[0].iloc[0]) == 'nan': # If the audio clip has at least one "basically". Note: "NaN" are given as entries in empty txt
#         print(name + "_" + str(id))
#         print(segment_ends[0])
        segment_ends[0].apply(np.float)
        segment_ends[1] = segment_ends[0].apply(sec_to_ms) # Create a new column of ms
        segment_ends = segment_ends[1].tolist() # List of end time segments 
        
        # Step 3: Insert ones
        #for segment_end in segment_ends:
        y = insert_ones_for_human_train_eg(y, segment_ends)
    
    audio = AudioSegment.from_wav(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav")
    audio = audio.set_frame_rate(123000)
    file_handle = audio.export(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav", format = "wav")

    sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + name + "_" + str(id) +".wav")
    frequencies, times, x = signal.spectrogram(samples, sample_rate)
    
    return frequencies, times, x, y

In [58]:
# Test case:
# frequencies, times, x, y = create_dev_example(44)
# c = 0
# for i in y[0]:
#     #print(i)
#     if (i == 1):
#        #print(i)
#        c += 1
# print(c)
#IPython.display.Audio(DEV_CUT_DIRECTORY + "cont_44.wav")

In [59]:
def create_human_train_X_Y(names,list_of_ids):
    """
    names -- list of names
    list_of_ids -- The list of ids of the desired dev audio files
    
    Returns:
    np.array(X)
    np.array(Y)
    """
    X, Y = [], []
    for name in names:
        for id in list_of_ids:
            frequencies, times, x, y = create_human_train_eg(name,id)
            x = np.transpose(x)
            y = np.transpose(y)
            print(name + "_" + str(id))
#             print(x)
#             print("shape of x: " + "x:{}".format(np.array(x).shape))
#             print(y)
            X.append(x)
            Y.append(y)
            print("shape of X: " + "X:{}".format(np.array(X).shape))
    X = np.array(X)
    Y = np.array(Y)
    return (X, Y)

## Create Numpy Arrays for Dev(Continuous) Set

In [60]:
def export_human_train_npy(names,list_of_ids):
    """
    Arguments:
    names -- list of names
    list_of_ids -- The list of ids of the desired dev audio files
    
    Function to save the numpy arrays of the dev audios to DEV_NPY_DIRECTORY
    """
    global GOOGLE_DATASET_BASED_NPY
    
    X, Y = create_human_train_X_Y(names,list_of_ids)
    np.save(GOOGLE_DATASET_BASED_NPY + "human_train_X.npy", X)
    np.save(GOOGLE_DATASET_BASED_NPY + "human_train_Y.npy", Y)
    print("Dimensions of np.array: " + "X:{} Y:{}".format(X.shape, Y.shape))
    print("np.array of X and Y + saved in " + GOOGLE_DATASET_BASED_NPY)

In [61]:
# A list of all currently available ids of dev audios
names = ["Chai", "larry", "yb", "aru", "jh"]
list_of_human_train_ids = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [62]:
export_human_train_npy(names, list_of_human_train_ids)

Chai_1
shape of X: X:(1, 5490, 129)
Chai_2
shape of X: X:(2, 5490, 129)
Chai_3
shape of X: X:(3, 5490, 129)
Chai_4
shape of X: X:(4, 5490, 129)
Chai_5
shape of X: X:(5, 5490, 129)
Chai_6
shape of X: X:(6, 5490, 129)
Chai_7
shape of X: X:(7, 5490, 129)
Chai_8
shape of X: X:(8, 5490, 129)
Chai_9
shape of X: X:(9, 5490, 129)
Chai_10
shape of X: X:(10, 5490, 129)
Chai_11
shape of X: X:(11, 5490, 129)
Chai_12
shape of X: X:(12, 5490, 129)
Chai_13
shape of X: X:(13, 5490, 129)
Chai_14
shape of X: X:(14, 5490, 129)
Chai_15
shape of X: X:(15, 5490, 129)
larry_1
shape of X: X:(16, 5490, 129)
larry_2
shape of X: X:(17, 5490, 129)
larry_3
shape of X: X:(18, 5490, 129)
larry_4
shape of X: X:(19, 5490, 129)
larry_5
shape of X: X:(20, 5490, 129)
larry_6
shape of X: X:(21, 5490, 129)
larry_7
shape of X: X:(22, 5490, 129)
larry_8
shape of X: X:(23, 5490, 129)
larry_9
shape of X: X:(24, 5490, 129)
larry_10
shape of X: X:(25, 5490, 129)
larry_11
shape of X: X:(26, 5490, 129)
larry_12
shape of X: X:(27, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



jh_2
shape of X: X:(62, 5490, 129)
Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-62-0c76d17cfbae>", line 1, in <module>
    export_human_train_npy(names, list_of_human_train_ids)
  File "<ipython-input-60-4ee0e101c667>", line 11, in export_human_train_npy
    X, Y = create_human_train_X_Y(names,list_of_ids)
  File "<ipython-input-59-580daece243c>", line 13, in create_human_train_X_Y
    frequencies, times, x, y = create_human_train_eg(name,id)
  File "<ipython-input-57-fdf6c1184b62>", line 22, in create_human_train_eg
    segment_ends[0].apply(np.float)
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\pandas\core\series.py", line 4042, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas\_libs\lib.pyx", line 2228, in pandas._libs.lib.map_infer
ValueError: could not c

ValueError: could not convert string to float: '1:53'

In [29]:
# Check Exported npy
print(np.load(GOOGLE_DATASET_BASED_NPY + "human_train_X.npy", allow_pickle=True))
# print(np.load(GOOGLE_DATASET_BASED_NPY + "human_train_Y.npy", allow_pickle=True))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-eaaa9ffda141>", line 2, in <module>
    print(np.load(GOOGLE_DATASET_BASED_NPY + "human_train_X.npy", allow_pickle=True))
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")
FileNotFoundError: [Errno 2] No such file or directory: './raw_data/google_dataset_based_npy/human_train_X.npy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Arushi Gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another excepti

FileNotFoundError: [Errno 2] No such file or directory: './raw_data/google_dataset_based_npy/human_train_X.npy'

In [32]:
# # Testing jh's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'jh' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

In [33]:
# # Testing larry's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'larry' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

In [34]:
# # Testing Chai's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'Chai' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

In [35]:
# Testing yb's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'yb' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)

In [ ]:
# Testing aru's .wav files
# sample_rate, samples = wavfile.read(DEV_GOOGLE_BASED_DATASET_CUT + 'aru' + "_" + str(1) +".wav")
# print(sample_rate)
# print(samples)
# print(len(samples))
# frequencies, times, x = signal.spectrogram(samples, sample_rate)
# print(np.unique(x))
# print(x)